# Named Entity Recognition using CRF model
Total per tag

* B-PER 2508
* I-PER 3111
* B-ADJ 402 
* I-ADJ 442
* B-ANM 2556
* I-ANM 2478
* B-GODS 467
* I-GODS 549
* B-OBJ 1661
* I-OBJ 986
* O 74768

#### Importing Libraries

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
# Membaca file 
df = pd.read_excel('ner_dataset.xlsx')

In [3]:
#Display first 10 rows
df.head(10)

,StoryID,Sentence,SentenceID,Word,Tag
0,0,anak_ririh,0,Pan,B-PER
1,0,anak_ririh,0,Karsa,I-PER
2,0,anak_ririh,0,ajaka,O
3,0,anak_ririh,0,pianakne,B-OBJ
4,0,anak_ririh,0,muani,I-OBJ
5,0,anak_ririh,0,nanggap,O
6,0,anak_ririh,0,upah,O
7,0,anak_ririh,0,ngae,O
8,0,anak_ririh,0,semer,O
9,0,anak_ririh,0,di,O


In [4]:
df.describe()

,StoryID,SentenceID
count,89917.000000,89917.000000
mean,80.035077,37.828308
std,50.116314,38.080827
min,0.000000,0.000000
25%,34.000000,13.000000
50%,71.000000,27.000000
75%,132.000000,50.000000
max,164.000000,289.000000


#### Observations : 
* There are total 47959 sentences in the dataset.
* Number unique words in the dataset are 35178.
* Total 17 lables (Tags).

In [5]:
#Displaying the unique Tags
df['Tag'].unique()

array(['B-PER', 'I-PER', 'O', 'B-OBJ', 'I-OBJ', 'B-ANM', 'I-ANM',
       'B-GODS', 'I-GODS', 'B-ADJ', 'I-ADJ'], dtype=object)

In [6]:
#Checking null values, if any.
df.isnull().sum()

StoryID       0
Sentence      0
SentenceID    0
Word          0
Tag           0
dtype: int64

There are lots of missing values in 'Sentence #' attribute. So we will use pandas fillna technique and use 'ffill' method which propagates last valid observation forward to next.

In [7]:
df = df.fillna(method = 'ffill')

In [8]:
# Memproses dataset untuk mengelompokkan per kalimat
class SentenceGetter:
    def __init__(self, data):
        self.data = data
        self.grouped = data.groupby("SentenceID").apply(
            lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                    s["Tag"].values.tolist())]
        )
        self.sentences = [sentence for sentence in self.grouped]

In [9]:
# Inisialisasi SentenceGetter
getter = SentenceGetter(df)
sentences = getter.sentences

# Menampilkan salah satu kalimat untuk verifikasi
print("Contoh kalimat pertama:", sentences[0])

Contoh kalimat pertama: [('Pan', 'B-PER'), ('Karsa', 'I-PER'), ('ajaka', 'O'), ('pianakne', 'B-OBJ'), ('muani', 'I-OBJ'), ('nanggap', 'O'), ('upah', 'O'), ('ngae', 'O'), ('semer', 'O'), ('di', 'O'), ('sisin', 'O'), ('rurunge', 'O'), ('gede', 'O'), ('.', 'O'), ('Ada', 'O'), ('katuturan', 'O'), ('satua', 'O'), ('I', 'B-ANM'), ('Angsa', 'I-ANM'), ('masawitra', 'O'), ('teken', 'O'), ('I', 'B-ANM'), ('Kerkuak', 'B-ANM'), ('.', 'O'), ('Kacerita', 'O'), ('Sang', 'B-GODS'), ('Hyang', 'I-GODS'), ('Indra', 'I-GODS'), ('kalintang', 'O'), ('sungsut', 'O'), ('santukan', 'O'), ('Ida', 'O'), ('mireng', 'O'), ('orti', 'O'), ('Indraloka', 'O'), ('pacang', 'O'), ('kagebugin', 'O'), ('olih', 'O'), ('Niwatakawaca', 'B-GODS'), ('ratun', 'O'), ('raksasane', 'O'), ('ring', 'O'), ('Manimantaka', 'O'), ('.', 'O'), ('Wenten', 'O'), ('kocap', 'O'), ('katuturan', 'O'), ('satua', 'O'), ('saking', 'O'), ('jagat', 'O'), ('Jembrana', 'O'), ('sane', 'O'), ('sampun', 'O'), ('lumbrah', 'O'), ('kabaosang', 'O'), ('antuk'

Getting all the sentences in the dataset.

#### Feature Preparation
These are the default features used by the NER in nltk. We can also modify it for our customization.

In [10]:
# Fungsi ekstraksi fitur dari setiap kata
# Hanya memproses word dan tag saja, postag dihapus

def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

# Ekstraksi fitur untuk satu kalimat
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# Ekstraksi label dari satu kalimat
def sent2labels(sent):
    return [label for token, label in sent]

# Ekstraksi token dari satu kalimat
def sent2tokens(sent):
    return [token for token, label in sent]


In [11]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [13]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [14]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

#### Evaluating the model performance.
We will use precision, recall and f1-score metrics to evaluate the performance of the model since the accuracy is not a good metric for this dataset because we have an unequal number of data points in each class.

In [15]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9539158236463694


In [16]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-ADJ       0.86      0.89      0.88        57
       B-ANM       0.90      0.83      0.87       461
      B-GODS       0.80      0.84      0.82        62
       B-OBJ       0.77      0.62      0.69       308
       B-PER       0.89      0.85      0.87       451
       I-ADJ       0.88      0.91      0.89        65
       I-ANM       0.89      0.90      0.89       436
      I-GODS       0.78      0.88      0.83        74
       I-OBJ       0.69      0.51      0.59       184
       I-PER       0.90      0.89      0.90       562
           O       0.97      0.98      0.98     12907

    accuracy                           0.96     15567
   macro avg       0.85      0.83      0.84     15567
weighted avg       0.95      0.96      0.95     15567



In [20]:
import joblib

# Menyimpan model
joblib.dump(crf, 'ner_crf_model.pkl')

['ner_crf_model.pkl']

This looks quite nice.

In [24]:
def test_model(sentence, crf_model):
    
    # Pecah kalimat menjadi kata-kata
    words = sentence.split()
    
    # Buat fitur untuk setiap kata (tanpa POS dalam contoh ini)
    sent_features = [
        {'bias': 1.0, 
         'word.lower()': word.lower(),
         'word[-3:]': word[-3:],
         'word[-2:]': word[-2:],
         'word.isupper()': word.isupper(),
         'word.istitle()': word.istitle(),
         'word.isdigit()': word.isdigit()} 
        for word in words
    ]
    
    # Tambahkan fitur konteks (kata sebelumnya dan sesudahnya)
    for i, word_features in enumerate(sent_features):
        if i > 0:  # Fitur kata sebelumnya
            word_features.update({
                '-1:word.lower()': words[i-1].lower(),
                '-1:word.istitle()': words[i-1].istitle(),
                '-1:word.isupper()': words[i-1].isupper(),
            })
        else:
            word_features['BOS'] = True  # Awal kalimat
        
        if i < len(words) - 1:  # Fitur kata sesudahnya
            word_features.update({
                '+1:word.lower()': words[i+1].lower(),
                '+1:word.istitle()': words[i+1].istitle(),
                '+1:word.isupper()': words[i+1].isupper(),
            })
        else:
            word_features['EOS'] = True  # Akhir kalimat

    # Prediksi tag menggunakan model
    predicted_tags = crf_model.predict([sent_features])[0]
    
    # Gabungkan kata dan tag untuk output
    result = list(zip(words, predicted_tags))
    return result


# Contoh penggunaan
kalimat_input = "Nuju sanja, I Rajapala ngaukin pianakné laut negak masila"
hasil = test_model(kalimat_input, crf)

# Tampilkan hasil
print("Hasil Prediksi:")
for kata, tag in hasil:
    print(f"{kata}\t{tag}")


Hasil Prediksi:
Nuju	O
sanja,	O
I	B-PER
Rajapala	I-PER
ngaukin	O
pianakné	O
laut	O
negak	O
masila	O


In [ ]:
import joblib

# Menyimpan model
joblib.dump(model, 'multinomial_nb_10 percent_model.pkl')